In [132]:
import glob, os, json
import solver
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

import time
import math

import traceback

In [146]:
np.arange(9, 11)

array([ 9, 10])

In [339]:
def generateData(rows, labels):
    inputs = np.empty(shape=(rows, labels))
    ordering = np.empty(shape=(rows, labels))
    for i in range(rows):
#         perm = np.random.permutation(np.arange(6, 6 + labels))
        perm = np.random.permutation(labels)
#         number = 0
#         for j in range(labels):
#             number += perm[j] * math.pow(10, labels - j - 1)
            
        ordering[i] = perm
        inputs[i] = perm[::-1]

#         inputs[i, 0] = number
#         inputs[i, 1] = 0
        
    return (inputs, ordering)

inputs, outputs = generateData(10000, 5)

In [340]:
inputs = normalize(inputs)
unnormalizedOutputs = outputs
outputs = normalize(outputs)

In [341]:
size = inputs.shape[0]
# Test data is separated in cleaning stage
trainSize = int(size * 0.75)
validSize = size - trainSize

inputsTrain = inputs[0:trainSize]
outputsTrain = outputs[0:trainSize]

inputsValid = inputs[trainSize:]
outputsValid = outputs[trainSize:]
unnormalizedOutputsValid = unnormalizedOutputs[trainSize:]

In [342]:
inputsTrain

array([[ 0.36514837,  0.        ,  0.54772256,  0.73029674,  0.18257419],
       [ 0.36514837,  0.73029674,  0.18257419,  0.        ,  0.54772256],
       [ 0.36514837,  0.73029674,  0.18257419,  0.54772256,  0.        ],
       ..., 
       [ 0.36514837,  0.73029674,  0.18257419,  0.54772256,  0.        ],
       [ 0.        ,  0.73029674,  0.36514837,  0.54772256,  0.18257419],
       [ 0.54772256,  0.        ,  0.18257419,  0.73029674,  0.36514837]])

In [268]:
# np.argsort(outputsTrain)[::-1]
# np.argsort(outputsTrain)[:,::-1]
np.argsort(outputsTrain)

array([[1, 0, 2, 4, 3],
       [3, 4, 0, 1, 2],
       [2, 4, 1, 3, 0],
       ..., 
       [4, 0, 2, 1, 3],
       [0, 2, 4, 3, 1],
       [3, 0, 4, 1, 2]])

In [165]:
with tf.Session().as_default():
    array1 = tf.constant([0, 1, 2, 3, 4], dtype=tf.float32)
    print(tf.nn.top_k(array1, k=5).values.eval())

[ 4.  3.  2.  1.  0.]


In [158]:
with tf.Session().as_default():
    prediction = tf.constant([10, 6, 9, 8, 7], dtype=tf.float32)
    actual = tf.constant([6, 7, 9, 8, 10], dtype=tf.float32)
    argsort_indices = tf.nn.top_k(prediction, k=5).indices
    print(argsort_indices.eval())
    print(tf.gather(actual, argsort_indices).eval())

[0 2 3 4 1]
[  6.   9.   8.  10.   7.]


In [88]:
np.argsort(outputsTrain[0])[::-1]

array([2, 0, 1, 4, 3])

In [138]:
normalize(np.array([0, 1, 2, 3, 4]).reshape(1, -1))

array([[ 0.        ,  0.18257419,  0.36514837,  0.54772256,  0.73029674]])

In [141]:
np.sum(normalize(np.array([0, 1, 2, 3, 4]).reshape(1, -1)))

1.8257418583505536

In [361]:
with tf.Session().as_default():
    array1 = tf.constant([ 1,  3,  2,  0,  4], dtype=tf.float64)
    array2 = tf.constant([ 1,  3,  2,  0,  4], dtype=tf.float64)
    print(ndcg(array1, array2).eval())

1.0


In [258]:
with tf.Session().as_default():
#     print(outputsTrain[])
#     print(tf.reduce_mean(ndcg(outputsTrain, outputsTrain)).eval())
    c = tf.map_fn(lambda x: ndcg(x[0], x[1]), (outputsTrain, outputsTrain), dtype=tf.float64)
    print(tf.reduce_mean(c).eval())
#     print(tf.gather(outputsTrain, tf.nn.top_k(outputsTrain, k=5).indices).eval())

1.0


In [289]:
EPOCHS = 10000

N1 = trainSize
LABEL_COUNT = 5

NODES1 = 512
NODES2 = 256

ALPHA = 0.08

BATCH_SIZE = 30

STD = 0.1

LEARNING_RATE = 0.1

In [290]:
# Define the input function for training
inputFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsTrain}, y=outputsTrain,
    batch_size=BATCH_SIZE, num_epochs=EPOCHS, shuffle=True)

In [291]:
# Define the neural network
def network(xDict):
    x = xDict["input"]
    
    flatten = tf.contrib.layers.flatten(x)
        
    regularizer = tf.contrib.layers.l2_regularizer(scale=ALPHA)
    
    # Hidden fully connected layer
#     layer1 = tf.layers.dense(flatten, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    
    layer1 = tf.layers.dense(flatten, NODES1, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Hidden fully connected layer
    layer2 = tf.layers.dense(flatten, 64, kernel_regularizer=regularizer, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    outLayer = tf.layers.dense(layer2, LABEL_COUNT)
    return outLayer

In [359]:
# Kullback-Leibler Divergence, as per https://stackoverflow.com/a/43298483
def klDivergence(p, q):
    pClipped = tf.clip_by_value(p, 1e-10, 1.0)
    qClipped = tf.clip_by_value(q, 1e-10, 1.0)
    return tf.reduce_sum(pClipped * tf.log(pClipped/qClipped))

# Loss function based off of Jensen-Shannon Divergence
def loss(label, prediction):
    mean = 0.5 * (label + prediction)
    return 0.5 * klDivergence(label, mean) + 0.5 * klDivergence(prediction, mean)

def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype))
    return numerator / denominator

# Accuracy metric using Normalized Discounted Cumulative Gain, as per https://github.com/shiba24/learning2rank/
def ndcg(labels, predictions, k=5):
    topK = tf.nn.top_k(labels, k=5)
    sortedValues = topK.values
    sortedIndices = topK.indices
#         print(labelSorted)
#         labelSorted = sorted(label, reverse=True)
    ideal_dcg = 0
    for i in range(k):
#             ideal_dcg += (2 ** labelSorted[:i] - 1.) / log2(tf.cast(i + 2, tf.float64))
        ideal_dcg += (sortedValues[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    dcg = 0
#         argsort_indices = np.argsort(predictions)[::-1]
#         argsort_indices = tf.nn.top_k(predictions, k=5).indices
#         print(argsort_indices)
    for i in range(k):
        dcg += (tf.gather(predictions, sortedIndices[i]) + 1) / log2(tf.cast(i + 2, tf.float64))
#         dcg += (predictions[i] + 1) / log2(tf.cast(i + 2, tf.float64))
    return dcg / ideal_dcg

In [386]:
# Define the model function (following TF Estimator Template)
def modelFunc(features, labels, mode):
    # Build the neural network
    logits = network(features)
    
#     resizedLogits = tf.reshape(logits, shape=[-1, MAX_SIZE * MAX_SIZE, 1])
    
    # Predictions
    # TODO: Possibly need to change
    pred_classes = logits
#     pred_classes = tf.argmax(logits, axis=1)
#     pred_probas = tf.nn.softmax(logits)
    pred_probas = tf.nn.sigmoid(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    print(logits.shape)
#     print(resizedLogits.shape)
    print(labels.shape)
    print(pred_classes.shape)
        
    # Define loss and optimizer
#     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#         logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
#     loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=logits, labels=labels))
    loss_op = tf.reduce_mean(loss(labels, logits))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
#     acc_op = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=pred_classes)
#     acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    ndcg_map = tf.map_fn(lambda x: ndcg(x[0], convertSinglePredToRank(x[1])), (labels, pred_classes), dtype=tf.float64)
    acc_op = tf.metrics.mean(ndcg_map)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
#       train_op=train_op)
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [387]:
# Build the Estimator
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
model = tf.estimator.Estimator(modelFunc, config=tf.contrib.learn.RunConfig(session_config=config))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c22a98128>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.5
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/2v/nktg94cn4cvfw3vprys2rgtm0000gn/T/tmp7yg7a69i'}


In [388]:
# Train the Model
model.train(inputFunc, steps=2000)

(?, 5)
(?, 5)
(?, 5)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/2v/nktg94cn4cvfw3vprys2rgtm0000gn/T/tmp7yg7a69i/model.ckpt.
INFO:tensorflow:loss = 77.3557845153, step = 1
INFO:tensorflow:global_step/sec: 757.805
INFO:tensorflow:loss = -26.5664104241, step = 101 (0.134 sec)
INFO:tensorflow:global_step/sec: 1102.01
INFO:tensorflow:loss = -26.5664104241, step = 201 (0.091 sec)
INFO:tensorflow:global_step/sec: 1123.43
INFO:tensorflow:loss = -26.5664104241, step = 301 (0.089 sec)
INFO:tensorflow:global_step/sec: 1077.52
INFO:tensorflow:loss = -26.5664104241, step = 401 (0.092 sec)
INFO:tensorflow:global_step/sec: 1170.14
INFO:tensorflow:loss = -26.5664104241, step = 501 (0.085 sec)
INFO:tensorflow:global_step/sec: 1186.46
INFO:tensorflow:loss = -26.5664104241, step = 601 (0.084 sec)
INFO:tensorflow:global_step/sec: 1176.71
INFO:tensorflow:loss = -26.5664104241, step = 701 (0.085 sec)
INFO:tensorflow:global_step/sec: 1001.27
INFO:te

In [389]:
# Evaluate the Model
# Define the input function for evaluating
validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid}, y=unnormalizedOutputsValid,
    batch_size=BATCH_SIZE, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(validFunc)

(?, 5)
(?, 5)
(?, 5)
INFO:tensorflow:Starting evaluation at 2018-03-01-20:05:46
INFO:tensorflow:Restoring parameters from /var/folders/2v/nktg94cn4cvfw3vprys2rgtm0000gn/T/tmp7yg7a69i/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-03-01-20:05:47
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.85962, global_step = 2000, loss = -3.42646e-08


{'accuracy': 0.85961992, 'global_step': 2000, 'loss': -3.4264641e-08}

In [371]:
validFunc = tf.estimator.inputs.numpy_input_fn(
    x={"input": inputsValid},
    batch_size=BATCH_SIZE, shuffle=False)
predictions = model.predict(validFunc)

In [372]:
predictions = np.array(list(predictions))

INFO:tensorflow:Restoring parameters from /var/folders/2v/nktg94cn4cvfw3vprys2rgtm0000gn/T/tmpmnvjs6my/model.ckpt-2000


In [373]:
predictions

array([[ 16.61369418,   9.71719172,  52.00292772,  10.6649038 ,
         16.69486406],
       [ 16.70020666,   9.89320897,  52.634171  ,  10.74836305,
         16.89704599],
       [ 16.65160719,   9.87297218,  52.27999647,  10.73323039,
         16.82406887],
       ..., 
       [ 16.97339922,  10.16850692,  53.58311961,  11.00491487,
         17.19138965],
       [ 16.92229189,  10.05589962,  53.27084152,  10.93379575,
         17.14208514],
       [ 16.43488798,   9.70844622,  51.66948251,  10.61542199,
         16.59278829]])

In [374]:
unnormalizedOutputsValid

array([[ 3.,  0.,  2.,  1.,  4.],
       [ 1.,  3.,  2.,  0.,  4.],
       [ 4.,  0.,  1.,  3.,  2.],
       ..., 
       [ 0.,  3.,  1.,  4.,  2.],
       [ 2.,  1.,  0.,  4.,  3.],
       [ 3.,  2.,  4.,  0.,  1.]])

In [375]:
with tf.Session().as_default():
#     print(outputsTrain[])
#     print(tf.reduce_mean(ndcg(outputsTrain, outputsTrain)).eval())
#     c = tf.map_fn(lambda x: ndcg(x[0], x[1]), (outputsValid, predictions), dtype=tf.float64)
#     print(tf.reduce_mean(c).eval())
    print(ndcg(outputsValid[0], predictions[0]).eval())

15.0457548582


In [376]:
def convertPredToRank(predictions):
    array = np.empty(shape=predictions.shape)
    for i, row in enumerate(predictions):
        array[i] = np.argsort(predictions[i])[::-1].reshape(1, -1).astype(float)
    return array

In [383]:
def convertSinglePredToRank(prediction):
    return tf.cast(tf.nn.top_k(prediction, k=5).indices, dtype=tf.float64)
#     return np.argsort(prediction)[::-1].reshape(1, -1).astype(float)

In [377]:
convertPredToRank(predictions)

array([[ 2.,  4.,  0.,  3.,  1.],
       [ 2.,  4.,  0.,  3.,  1.],
       [ 2.,  4.,  0.,  3.,  1.],
       ..., 
       [ 2.,  4.,  0.,  3.,  1.],
       [ 2.,  4.,  0.,  3.,  1.],
       [ 2.,  4.,  0.,  3.,  1.]])

In [378]:
with tf.Session().as_default():
    c = tf.map_fn(lambda x: ndcg(x[0], x[1]), (unnormalizedOutputsValid, convertPredToRank(predictions)), dtype=tf.float64)
    print(tf.reduce_mean(c).eval())

0.862375412267


In [379]:
with tf.Session().as_default():
    print(ndcg(unnormalizedOutputsValid[1], convertPredToRank(predictions)[1]).eval())

0.826922872058


In [344]:
np.argsort(predictions[0])[::-1].reshape(1, -1).astype(float)

array([[ 2.,  3.,  1.,  0.,  4.]])

In [385]:
with tf.Session().as_default():
    array1 = tf.constant([0, 4, 1, 3, 2], dtype=tf.float64)
    print(convertSinglePredToRank(array1).eval())

[ 1.  3.  4.  2.  0.]
